# 따릉이 고장데이터 전처리

<br>
<br>

[1. 필요모듈 및 전역변수](#1.-필요모듈-및-전역변수)

<br>

[2. 데이터 로드](#2.-데이터-로드)
<br>
[3. 데이터 전처리](#3.-데이터-전처리)
<br>
[4. 인사이트를 위한 전처리](#4.-인사이트를-위한-전처리)
<br>

1~3까지는 필수로 순서대로 실행하셔야합니다! 4부터는 원하는 버전을 실행하셔도 됩니다.
<br>
데이터가 너무 많아 실행시간이 너무 깁니다. 12월 데이터만 사용하는게 좋을 것 같습니다.

## 1. 필요모듈 및 전역변수

`START_MONTH` ~ `END_MONTH` 까지의 데이터를 사용합니다. 둘 다 12를 권장합니다

In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import random as rd
from IPython.display import clear_output
from datetime import timedelta

START_MONTH = 12 # 7~12 
END_MONTH = 12 # 12월로 고정바람
DAY_NUM = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
rent_file_name = ["", "", "", "", "", "", "",
                  "서울특별시 공공자전거 대여이력 정보_2207.csv", 
                  "서울특별시 공공자전거 대여이력 정보_22081.csv",
                  "서울특별시 공공자전거 대여이력 정보_2209.csv",
                  "서울특별시 공공자전거 대여이력 정보_2210.csv", 
                  "서울특별시 공공자전거 대여이력 정보_2211.csv",
                  "서울특별시 공공자전거 대여이력 정보_2212.csv"]

<br>

## 2. 데이터 로드

대여이력과 고장신고데이터를 로드합니다

In [2]:
# 데이터 로드

print("데이터 로드중..")
df_rent = [None] * 13
for i in range(START_MONTH, END_MONTH + 1):
    df_rent[i] = pd.read_csv(rent_file_name[i], encoding='cp949')
    print(f"{i}월 대여이력 로드 완료")

df_broken = pd.read_csv("서울시 공공자전거 고장신고 내역_22.07-12.csv", encoding='cp949')
print("고장데이터 로드 완료")

데이터 로드중..
12월 대여이력 로드 완료
고장데이터 로드 완료


<br>

## 3. 데이터 전처리

중복되는 정보를 담은 열과 데이터형식을 알맞게 전처리합니다.

In [3]:
def 대여이력정보_전처리(df):
    # 시계열 데이터 전처리
    df = df[(df['대여일시'] != '0000-00-00 00:00:00') & (df['반납일시'] != '0000-00-00 00:00:00')]
    df['대여일시'] = pd.to_datetime(df['대여일시']) # 문자열로 되있는 날짜들을 datatime 타입으로 변경
    df['대여요일'] = df['대여일시'].dt.day_name()
    df['반납일시'] = pd.to_datetime(df['반납일시'])
    
    # 반납이 되지 않아 대여소 번호가 없으면 -1로 전처리
    df["반납대여소번호"] = df["반납대여소번호"].str.replace('\\','')
    df["반납대여소번호"] = df["반납대여소번호"].str.replace('N','-1')
    df = df.astype({'반납대여소번호': 'int'})
    
     # 중복되어 필요없는 열 삭제
    df.drop(['대여거치대', '반납거치대', '대여대여소ID', '반납대여소ID'], axis=1, inplace=True)
    return df

def 고장신고내역_전처리(df):
    df['등록일시'] = pd.to_datetime(df['등록일시']) # 문자열로 되있는 날짜들을 datatime 타입으로 변경
    df = df.drop_duplicates(['자전거번호', '등록일시']) # 같은 고장신고인데 유형만 다른 데이터 중복 제거
    return df

print("데이터 전처리중..")
df_broken_p = 고장신고내역_전처리(df_broken)
print("고장신고내역 전처리 완료")

# 대여이력 전처리 실행
df_rent_m = [None] * 13 # 월 기준으로 나눈 대여이력 : df_rent_m[12] -> 12월 대여이력 df 
for m in range(START_MONTH, END_MONTH + 1):
    df_rent_m[m] = 대여이력정보_전처리(df_rent[m])
    print(f"{m}월 대여이력 전처리 완료")

데이터 전처리중..
고장신고내역 전처리 완료


C:\Users\imihl\AppData\Local\Temp\ipykernel_21248\4165522044.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["반납대여소번호"] = df["반납대여소번호"].str.replace('\\','')


12월 대여이력 전처리 완료


<br>

추가로, 1달짜리 데이터를 한번에 사용하면 탐색시간이 너무 길므로 **일(day) 단위**로 나눠 탐색하기위해, 데이터를 **일 단위**로 분할합니다

In [4]:
# 필요한 날짜만 조회하여 시간을 줄이기 위해 일 단위로 df 나누기
df_rent_d = [[None] * 32 for i in range(13)] # 월, 일 기준으로 나눈 대여이력 : df_rent_d[12][25] -> 12월 25일 대여이력 df
for m in range(START_MONTH, END_MONTH + 1):
    now = datetime.datetime(2022, m, 1)
    now = now.replace(hour=0, minute=0, second=0)
    next_day = now + timedelta(hours=24)
    
    for d in range(1, DAY_NUM[m] + 1):
        df_rent_d[m][d] = df_rent_m[m][(now <= df_rent_m[m]['대여일시']) & (df_rent_m[m]['대여일시'] < next_day)]
        now += timedelta(hours=24)
        next_day += timedelta(hours=24)

print("대여이력 일별로 분할 완료")

대여이력 일별로 분할 완료


<br>

## 4. 인사이트를 위한 전처리

기존 파일에 쓸만한 새로운 행들을 추가하는 작업입니다.

단계적으로 개발하며 여러 버전을 만들었습니다.

아마 ver. 3 고장유무 라벨링을 제일 많이 사용할 것 같습니다.

매개변수를 조절해가며 여러 인사이트들을 얻을 수 있을 것 같습니다.
<br>

## ver. 1 : 고장신고 되기 N시간 전 ~ 고장신고까지 정보 추출

### 기능

두 데이터프레임(csv파일)을 생성합니다

**`기능1(첫번째 리턴값)`** : 기존 고장신고내역에 **고장신고 되기 N시간 전 ~ 고장신고** 까지의 대여 횟수를 추가하고 새로운 csv파일에 저장합니다.
<br>
**`기능2(두번째 리턴값)`** : 고장난 자전거들의 **고장신고 되기 N시간 전 ~ 고장신고**까지의 모든 대여이력을 모아 새로운 csv파일에 저장합니다. 즉, 고장 데이터들만 모여 있습니다.

### 매개변수
**`n`** : 고장신고일시로부터 n시간 전까지 고장으로 분류합니다
<br>
**`start_month`** : 데이터의 시작월

In [11]:
def 고장신고부터_n시간전까지_정보추출(n, start_month = 12):
    rent_num = [] # n시간동안 대여횟수
    broken_id = [] # 원본 파일 인덱스
    broken_date = [] # 고장 일시
    
    df_new_broken = pd.DataFrame(columns=df_broken_p.columns)
    df_rent_broken = pd.DataFrame(columns = df_rent_m[start_month].columns)
    
    start_time = time.time()
    for i, row in df_broken_p.iterrows():
        end = row['등록일시']
        start = end - timedelta(hours=n)
        
        # start_month 월 이전 데이터를 사용해야하는 고장신고는 건너뛰기
        if start.month < start_month:
            continue

        filtered_df = pd.DataFrame(columns = df_rent_m[start_month].columns)
        # start와 end month가 다를수도 있으므로 해당 기간에 속한 month들 모두 탐색
        for m in range(start.month, end.month + 1):
            end_day = end.day if m == end.month else DAY_NUM[m]
            start_day = start.day if m == start.month else 1
            for d in range(start_day, end_day + 1):
            # 기간안에 속한 대여이력 추출
                tmp_df = df_rent_d[m][d][(start <= df_rent_d[m][d]['대여일시']) & (df_rent_d[m][d]['대여일시'] < end)]
                tmp_df = tmp_df[tmp_df['자전거번호'] == row[0]]
                filtered_df = pd.concat([tmp_df, filtered_df])
                

        # 등록일시와 고장신고인덱스 정보 추가
        rent_num.append(len(filtered_df))
        broken_id.append(i)
        broken_date.append(row['등록일시'])
        df_new_broken = pd.concat([df_new_broken, row.to_frame().T], ignore_index=True)
        
        filtered_df['고장등록일시'] = row['등록일시']
        filtered_df['고장신고인덱스'] = i
        df_rent_broken = pd.concat([df_rent_broken, filtered_df], ignore_index=True)
        
        #진행도 출력
        if i % 100 == 0:
            clear_output()
            now = time.time() - start_time
            print("{:.2f}%... {:.2f}초 경과".format((i - broken_id[0]) / (len(df_broken) - broken_id[0]) * 100, now))
        
        
    print(f"100% 완료!... {time.time() - start_time}초 경과")
    
    name = str(n) + "시간동안대여수"
    df_new_broken[name] = rent_num
    df_new_broken["원본파일인덱스"] = broken_id
    
    name = f"{start_month}~12 고장신고내역(N={n}시간).csv"
    df_new_broken.to_csv(name, encoding='cp949')
    name = f"{start_month}~12 대여이력(N={n}시간내고장).csv"
    df_rent_broken.to_csv(name, encoding='cp949')
    print("파일생성완료")
    
    return df_new_broken, df_rent_broken

df, df2 = 고장신고부터_n시간전까지_대여이력추출(72, START_MONTH)

98.01%... 495.81초 경과
100% 완료!... 505.7469012737274초 경과
파일저장완료


<br>

## ver. 2 : ver. 1에 더해 짧은 대여 이력 정보추출


### 기능

두 데이터프레임(csv파일)을 생성합니다

**`기능1(첫번째 리턴값)`** : ver.1 의 기능1에 더해 짧은대여 정보를 추가했습니다.
<br>
**`기능2(두번째 리턴값)`** : 고장난 자전거들의 **고장신고 되기 N시간 전 ~ 고장신고**까지의 모든 **짧은대여이력**만을 모아 새로운 csv파일에 저장합니다. 즉, 고장 데이터들중 짧은 대여이력만 모여 있습니다.

### 매개변수
**`n`** : 고장신고일시로부터 n시간 전까지 고장으로 분류합니다
<br>
**`short_dist`** : 짧은 대여의 기준 거리 -> short_dist 이하의 이용거리들을 짧은 대여라고 판단합니다.
<br>
**`short_time`** : 짧은 대여의 기준 시간 -> short_dist 이하의 대여시간들을 짧은 대여라고 판단합니다.
<br>
**`start_month`** : 데이터의 시작월

- 거리와 시간 중 하나라도 기준 이하면 짧은 거리라고 판단합니다

In [12]:
def 고장신고부터_n시간전까지_짧은대여정보추출(n, short_dist = 100, short_time = 1, start_month = 12):
    rent_num = [] # n시간동안 대여횟수
    short_rent_num = [] # n시간동안 짧은 대여 횟수
    broken_id = [] # 원본 파일 인덱스
    pro = [] # 짧은대여수 / 총 대여수
    broken_date = []
    
    df_new_broken = pd.DataFrame(columns=df_broken_p.columns)
    df_rent_broken = pd.DataFrame(columns = df_rent_m[start_month].columns)
    
    start_time = time.time()
    for i, row in df_broken_p.iterrows():
        end = row['등록일시']
        start = end - timedelta(hours=n)
        
        # start_month 월 이전 데이터를 사용해야하는 고장신고는 건너뛰기
        if start.month < start_month:
            continue

        filtered_df = pd.DataFrame(columns = df_rent_m[start_month].columns)
        # start와 end month가 다를수도 있으므로 해당 기간에 속한 month들 모두 탐색
        for m in range(start.month, end.month + 1):
            end_day = end.day if m == end.month else DAY_NUM[m]
            start_day = start.day if m == start.month else 1
            for d in range(start_day, end_day + 1):
            # 기간안에 속한 대여이력 추출
                tmp_df = df_rent_d[m][d][(start <= df_rent_d[m][d]['대여일시']) & (df_rent_d[m][d]['대여일시'] < end)]
                tmp_df = tmp_df[tmp_df['자전거번호'] == row[0]]
                filtered_df = pd.concat([tmp_df, filtered_df])
        rent_num.append(len(filtered_df))
        
        
        # 짧은 대여 이력 추출 - 거리와 시간 중 하나라도 만족하면 짧은 거리라고 판단
        filtered_df = filtered_df[(filtered_df['이용거리(M)'] <= short_dist) | (filtered_df['이용시간(분)'] <= short_time)]
        short_rent_num.append(len(filtered_df))
        pro.append(0 if rent_num[-1] == 0 else short_rent_num[-1] / rent_num[-1])

        broken_id.append(i)
        broken_date.append(row['등록일시'])
        df_new_broken = pd.concat([df_new_broken, row.to_frame().T], ignore_index=True)
        
        filtered_df['고장일시'] = row['등록일시']
        df_rent_broken = pd.concat([df_rent_broken, filtered_df], ignore_index=True)
        
        if i % 100 == 0:
            clear_output()
            now = time.time() - start_time
            print("{:.2f}%... {:.2f}초 경과".format((i - broken_id[0]) / (len(df_broken) - broken_id[0]) * 100, now))
        
        
    print(f"100% 완료!... {time.time() - start_time}초 경과")
    
    name = str(n) + "시간동안대여수"
    df_new_broken[name] = rent_num
    df_new_broken["짧은대여수"] = short_rent_num
    df_new_broken["짧은대여비율"] = pro
    df_new_broken["원본파일인덱스"] = broken_id
    
    name = f"{start_month}~12 고장신고내역+짧은대여(N={n}시간,dist={short_dist},time={short_time}).csv"
    df_new_broken.to_csv(name, encoding='cp949')
    name = f"{start_month}~12 짧은대여이력(N={n}시간,dist={short_dist},time={short_time}).csv"
    df_rent_broken.to_csv(name, encoding='cp949')
    print("파일생성완료")
    
    return df_new_broken, df_rent_broken

df, df2 = 고장신고부터_n시간전까지_대여이력추출(72, start_month=START_MONTH)

98.01%... 473.43초 경과
100% 완료!... 483.7968490123749초 경과
파일저장완료


<br>

## ver. 3 : 고장유무 라벨링

대여이력에서 고장 유무를 알 수 있게 됨으로써 수월한 데이터 분석이 가능할 것 같습니다

### 기능

하나의 데이터프레임(csv파일)을 생성합니다.

**기존 대여이력에 고장유무 열과 해당 고장이 고장신고 파일에서 어느 고장인지 나타내는 고장신고인덱스를 추가합니다.**

컴퓨터에 따라 메모리 초과의 위험이 있습니다. 만약 3분내로 `"데이터 로드 완료. 고장 데이터 탐색중"`이 출력되지 않으면 메모리 초과일 확률이 높습니다.

메모리 초과가 발생하면 주피터 노트북을 재시작하여 `3. 데이터 전처리`까지만 실행하고 `ver.1`, `ver.2`는 건들지 마시고 바로 `ver.3`을 실행해보세요.

### 매개변수
**`n`** : 고장신고일시로부터 n시간 전까지 고장으로 분류합니다
<br>
**`month`** : 라벨링할 하나의 월 (다수의 월을 동시에 하면 시간도 시간이지만 메모리초과로 주피터가 작동불능)

In [8]:
def 고장유무_라벨링(n, month):
    고장유무, 고장신고인덱스 = 0, 1
    #고장등록일시 = 2
    
    start_time = time.time()
    is_broken = {}
    
    print("데이터 로드중")
    bike_id = np.concatenate((df_rent_m[month]['자전거번호'].unique(), df_broken_p['자전거번호'].unique()), axis = 0)
    for id in bike_id:
        is_broken[id] = [[[False] * 2 for __ in range(24)] for _ in range(DAY_NUM[month] + 1)]
    
    print("데이터 로드 완료. 고장 데이터 탐색중")
    for i, row in df_broken_p.iterrows():
        if row['등록일시'].month != month:
            continue
            
        end = row['등록일시']
        start = end - timedelta(hours=n)
        
        # start가 이전월로 넘어가버리면 start를 그 월의 시작으로 설정
        if start.month < month:
            start = datetime.datetime(2022, month, 1)
            start = start.replace(hour=0, minute=0, second=0)
            
         # start부터 end까지는 고장 True로 설정
        for d in range(start.day, end.day + 1):
            end_hour = end.hour if d == end.day else 23
            start_hour = start.hour if d == start.day else 0
            for h in range(start_hour, end_hour + 1):
                is_broken[row['자전거번호']][d][h][고장유무] = True
                is_broken[row['자전거번호']][d][h][고장신고인덱스] = i
        
        #진행도 출력
        if i % 10000 == 0:
            clear_output()
            now = time.time() - start_time
            print("고장 데이터 탐색중... {:.2f}%... {:.2f}초 경과".format(i / len(df_broken) * 100, now))
    
    
    # 대여이력의 대여일시를 순회하며 is_broken[대여일시]이 True이면 고장여부 1로 설정
    df_new_rent = df_rent_m[month]
    df_new_rent['고장여부'] = 0
    df_new_rent['고장신고인덱스'] = 0
    for i, row in df_rent_m[month].iterrows():
        if is_broken[row['자전거번호']][row['대여일시'].day][row['대여일시'].hour][고장유무]:
            df_new_rent.loc[i, '고장여부'] = 1
            df_new_rent.loc[i, '고장신고인덱스'] = is_broken[row['자전거번호']][row['대여일시'].day][row['대여일시'].hour][1]
            
        #진행도 추력
        if i % 10000 == 0:
            clear_output()
            now = time.time() - start_time
            print("대여이력 탐색중... {:.2f}%... {:.2f}초 경과".format(i / len(df_rent_m[month]) * 100, now))
    
    print("100% 탐색완료. 파일 생성중")
    name = f"{month}월 대여이력 정보(고장여부추가).csv"
    df_new_rent.to_csv(name, encoding='cp949')
    print("파일생성완료")
    return df_new_rent

df = 고장유무_라벨링(72, 12)
df.head()

대여이력 탐색중... 99.70%... 673.22초 경과
100%! 파일 생성중
파일생성완료


,자전거번호,대여일시,대여 대여소번호,대여 대여소명,반납일시,반납대여소번호,반납대여소명,이용시간(분),이용거리(M),생년,성별,이용자종류,대여요일,고장여부,고장신고인덱스
0,SPB-44695,2022-12-01 00:00:10,1933,개봉푸르지오아파트 상가,2022-12-01 00:00:20,1933,개봉푸르지오아파트 상가,0,0.00,\N,M,내국인,Thursday,0,0
1,SPB-31562,2022-12-01 00:00:04,3007,MBC 앞,2022-12-01 00:00:26,3007,MBC 앞,0,111.20,1995,F,내국인,Thursday,0,0
2,SPB-56324,2022-12-01 00:01:33,4468,가락1동주민센터,2022-12-01 00:01:58,4468,가락1동주민센터,0,0.00,1994,M,내국인,Thursday,0,0
3,SPB-30175,2022-12-01 00:03:02,652,답십리 래미안엘파인아파트 입구,2022-12-01 00:03:30,652,답십리 래미안엘파인아파트 입구,0,0.00,1981,M,내국인,Thursday,0,0
4,SPB-37639,2022-12-01 00:00:18,1047,강동 한신휴플러스,2022-12-01 00:03:55,1075,천동초교 삼거리,3,492.34,1989,M,내국인,Thursday,0,0
